In [ ]:
!pip install pandas requests pycountry tqdm
!export SHIPSTATION_API_KEY='TEST_R5ryzjmu4AFiFq33pfYZ8wZHmhW6CCXME2IZ6PqZv1k'

Getting Shipping rates from Shipstation API (old method that possible could change once they transition from ShipEngine)

In [ ]:
import os, requests, pandas as pd, pycountry, time
from tqdm.auto import tqdm

BASE   = "https://api.shipstation.com/v2"
HEAD   = {"API-Key": "TEST_R5ryzjmu4AFiFq33pfYZ8wZHmhW6CCXME2IZ6PqZv1k",
          "Content-Type": "application/json"}

def iso(cc_name:str)->str:
    """Convert full country name → ISO-3166-2 code (ShipStation needs this)."""
    return pycountry.countries.lookup(cc_name).alpha_2

def pounds(kg:float)->float:
    return kg * 2.20462

def get_carrier_id(code="fedex"):
    r = requests.get(f"{BASE}/carriers", headers=HEAD)
    r.raise_for_status()
    data = r.json()
    carriers = data["carriers"] if isinstance(data, dict) else data
    try:
        return next(c["carrier_id"] for c in carriers
                    if c.get("code", "").lower().startswith(code.lower()))
    except StopIteration:
        raise RuntimeError(f"No carrier with code starting '{code}' found. "
                           "Is it connected in ShipStation?")

FEDEX_ID = get_carrier_id()
print("FedEx carrier_id:", FEDEX_ID)


debugging for 401/403 errors

In [ ]:
carriers_resp = requests.get(f"{BASE}/carriers", headers=HEAD)
carriers_resp.raise_for_status() # will raise on 401/403
carriers_json = carriers_resp.json()

sending API requests for lowest rate from FedEx

In [ ]:
def lowest_fedex_rate(frm_country, to_country, tot_weight_kg):
    body = {
        "rate_options": {"carrier_ids": [FEDEX_ID]},
        "shipment": {
            "validate_address": "no_validation",
            "ship_from": {
                "name": "Warehouse",
                "address_line1": "N/A",
                "city_locality": "N/A",
                "state_province": "N/A",
                "postal_code": "00000",
                "country_code": iso(frm_country),
                "address_residential_indicator": "unknown"
            },
            "ship_to": {
                "name": "Consignee",
                "address_line1": "N/A",
                "city_locality": "N/A",
                "state_province": "N/A",
                "postal_code": "00000",
                "country_code": iso(to_country),
                "address_residential_indicator": "unknown"
            },
            "packages": [
                {
                    "package_code": "package",
                    "weight": {"value": round(pounds(tot_weight_kg), 3),
                               "unit": "pound"}
                }
            ]
        }
    }
    r = requests.post(f"{BASE}/rates", headers=HEAD, json=body)
    r.raise_for_status()
    rates = r.json()["rate_response"]["rates"]
    return min(rate["shipping_amount"]["amount"] for rate in rates)


adding as a column to PAHO dataset

In [ ]:
df = pd.read_csv("PAHO_data.csv")
def _get(row):
    try:
        return lowest_fedex_rate(row["SupplierCountry"],
                                 row["Country"],
                                 row["TotalWeight"])
    except Exception as e:
        print(f"Rate error on row {row.name}: {e}")
        return None

tqdm.pandas()
df["FedexAllPred"] = df.progress_apply(_get, axis=1)

In [ ]:
import pandas as pd
import numpy as np
from ngboost import NGBRegressor
from ngboost.distns import Normal
from ngboost.scores import MLE
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

data cleaning

In [ ]:
df["PO_Date"]   = pd.to_datetime(df["PurchaseOrderDate"])
df["PO_year"]   = df["PO_Date"].dt.year
df["PO_month"]  = df["PO_Date"].dt.month
df["PO_day"]    = df["PO_Date"].dt.day
df = df.drop(columns=["PurchaseOrderDate", "PO_Date"])

splitting numeric and labeled categories

In [ ]:
target = "Freight"

numeric_cols = [
    "FedexAllPred", "Quantity", "Value", "Air", "BDI", "Doses",
    "Distance", "Volume", "Openness", "Weight", "TotalWeight",
    "PO_year", "PO_month", "PO_day"
]

categorical_cols = [
    "Country", "SupplierCountry", "Type", "Product",
    "Variant", "StorageCondition"
]

running NGBoost

In [ ]:
X = df[numeric_cols + categorical_cols]
y = df[target]

numeric_tf   = Pipeline([("scale", StandardScaler())])
categorical_tf = Pipeline([("ohe", OneHotEncoder(handle_unknown="ignore"))])

preprocess = ColumnTransformer(
    [("num", numeric_tf, numeric_cols),
     ("cat", categorical_tf, categorical_cols)]
)

ngb = NGBRegressor(
    Dist=Normal,
    Score=MLE(),
    n_estimators=800,
    learning_rate=0.03,
    verbose=True,
    random_state=0
)

model = Pipeline([
    ("prep", preprocess),
    ("ngb",  ngb)
])

testing and training splits

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

fitting

In [ ]:
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae    = mean_absolute_error(y_test, y_pred)
rmse   = mean_squared_error(y_test, y_pred, squared=False)

print(f"MAE  : {mae:,.2f}")
print(f"RMSE : {rmse:,.2f}")

y_dists = model.predict_dist(X_test)
ci95_lo = y_dists.dist.ppf(0.025)
ci95_hi = y_dists.dist.ppf(0.975)

FAILED Attempt: Use the FedEx Web Services (Rate API)

In [ ]:
!pip install python-fedex


In [ ]:
from fedex.config import FedExConfig
from fedex.services.rate_service import FedExRateServiceRequest

# 1) point to your FedEx credential file / values
config = FedExConfig(
    key='YOUR_KEY',
    password='YOUR_PASSWORD',
    account_number='YOUR_ACCOUNT',
    meter_number='YOUR_METER',
    use_test_server=True
)

def get_fedex_rate(origin_country, dest_country, weight_lbs, length, width, height):
    if weight_lbs <= 0:
        return 0.0
    rate_request = FedExRateServiceRequest(config)
    rate_request.RequestedShipment.DropoffType = 'REGULAR_PICKUP'
    rate_request.RequestedShipment.PackagingType = 'YOUR_PACKAGE_TYPE'
    # shipper
    rate_request.RequestedShipment.Shipper.Address.CountryCode = origin_country
    # recipient
    rate_request.RequestedShipment.Recipient.Address.CountryCode = dest_country
    # package
    pkg = rate_request.create_wsdl_object_of_type('RequestedPackageLineItem')
    pkg.Weight.Value = weight_lbs
    pkg.Weight.Units = 'LB'
    pkg.Dimensions.Length = length
    pkg.Dimensions.Width = width
    pkg.Dimensions.Height = height
    pkg.Dimensions.Units = 'IN'
    rate_request.add_package(pkg)

    rate_request.send_request()
    svc = rate_request.HighestSeverity == 'SUCCESS' and rate_request.RateReply.RateReplyDetails[0]
    return float(svc.RatedShipmentDetails[0].ShipmentRateDetail.TotalNetChargeWithDutiesAndTaxes.Amount)


In [ ]:
import pandas as pd

df = pd.read_csv('your_data.csv')
# choose which weight to use
df['use_weight'] = df[['Weight', 'TotalShipmentWeight']].max(axis=1)
# fill zeros → skip API calls
df['FedExRate'] = df.apply(lambda r: get_fedex_rate(
    origin_country   = r['SupplierCountry'],
    dest_country     = r['Country'],
    weight_lbs       = r['use_weight'],
    length           = 10,
    width            = 10,
    height           = 10
), axis=1)
df.to_csv('with_fedex_rates.csv', index=False)
